In [6]:
import polars as pl
from geopy.geocoders import Nominatim
import time

In [2]:
df = pl.read_csv("nsw_property_data.csv")
print(f"Loaded {len(df)} rows from nsw_property_data.csv")

Loaded 4854814 rows from nsw_property_data.csv


In [3]:
df

property_id,download_date,council_name,purchase_price,address,post_code,property_type,strata_lot_number,property_name,area,area_type,contract_date,settlement_date,zoning,nature_of_property,primary_purpose,legal_description
i64,str,str,i64,str,i64,str,i64,str,f64,str,str,str,str,str,str,str
1672682,"""2024-02-19""","""BYRON""",1120000,"""129 MAFEKING RD, GOONENGERRY""",2482,"""house""",null,null,3.419,"""H""","""2024-01-19""","""2024-02-15""","""RU2""","""R""","""RESIDENCE""","""1/607703"""
1665192,"""2024-02-19""","""BYRON""",2800000,"""38 AVOCADO CRES, EWINGSDALE""",2481,"""house""",null,null,3420.0,"""M""","""2023-11-10""","""2024-02-15""","""R5""","""R""","""RESIDENCE""","""25/806200"""
1676023,"""2024-02-19""","""BYRON""",1500000,"""20 ROYAL AVE, SOUTH GOLDEN BEA…",2483,"""house""",null,null,727.2,"""M""","""2023-11-17""","""2024-02-09""","""R2""","""R""","""RESIDENCE""","""213/31166"""
1670957,"""2024-02-19""","""BYRON""",985000,"""2 INDERWONG AVE, OCEAN SHORES""",2483,"""house""",null,null,992.7,"""M""","""2023-09-27""","""2024-02-12""","""R2""","""R""","""RESIDENCE""","""1101/241074"""
1673846,"""2024-02-19""","""BYRON""",720000,"""52 NAROOMA DR, OCEAN SHORES""",2483,"""house""",null,null,771.4,"""M""","""2024-02-06""","""2024-02-09""","""R2""","""R""","""RESIDENCE""","""337/238455"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3137396,"""2004-12-16""","""WOLLONGONG""",190000,"""1/186 BALGOWNIE RD, BALGOWNIE""",2519,"""unit""",1,null,null,null,"""2004-05-27""","""2004-11-29""",null,"""R""","""RESIDENCE""","""1/SP73961"""
3137396,"""2004-12-16""","""WOLLONGONG""",190000,"""2/186 BALGOWNIE RD, BALGOWNIE""",2519,"""unit""",2,null,null,null,"""2004-05-27""","""2004-11-29""",null,"""R""","""RESIDENCE""","""2/SP73961"""
3138813,"""2004-12-16""","""WOLLONGONG""",380000,""" HAWTHORN ST, TARRAWANNA""",2518,"""house""",null,null,21.17,"""H""","""2003-12-15""","""2004-06-25""",null,"""V""","""VACANT LAND""","""2/793302"""


In [4]:
unique_councils = df.select("council_name").unique().sort("council_name")
print("Unique council names:")
print(unique_councils)

Unique council names:
shape: (220, 1)
┌───────────────┐
│ council_name  │
│ ---           │
│ str           │
╞═══════════════╡
│ 003           │
│ 011           │
│ 013           │
│ 014           │
│ 020           │
│ …             │
│ WINGECARRIBEE │
│ WOLLONDILLY   │
│ WOLLONGONG    │
│ WOOLLAHRA     │
│ YASS VALLEY   │
└───────────────┘


In [5]:
df.head()

property_id,download_date,council_name,purchase_price,address,post_code,property_type,strata_lot_number,property_name,area,area_type,contract_date,settlement_date,zoning,nature_of_property,primary_purpose,legal_description
i64,str,str,i64,str,i64,str,i64,str,f64,str,str,str,str,str,str,str
1672682,"""2024-02-19""","""BYRON""",1120000,"""129 MAFEKING RD, GOONENGERRY""",2482,"""house""",null,null,3.419,"""H""","""2024-01-19""","""2024-02-15""","""RU2""","""R""","""RESIDENCE""","""1/607703"""
1665192,"""2024-02-19""","""BYRON""",2800000,"""38 AVOCADO CRES, EWINGSDALE""",2481,"""house""",null,null,3420.0,"""M""","""2023-11-10""","""2024-02-15""","""R5""","""R""","""RESIDENCE""","""25/806200"""
1676023,"""2024-02-19""","""BYRON""",1500000,"""20 ROYAL AVE, SOUTH GOLDEN BEA…",2483,"""house""",null,null,727.2,"""M""","""2023-11-17""","""2024-02-09""","""R2""","""R""","""RESIDENCE""","""213/31166"""
1670957,"""2024-02-19""","""BYRON""",985000,"""2 INDERWONG AVE, OCEAN SHORES""",2483,"""house""",null,null,992.7,"""M""","""2023-09-27""","""2024-02-12""","""R2""","""R""","""RESIDENCE""","""1101/241074"""
1673846,"""2024-02-19""","""BYRON""",720000,"""52 NAROOMA DR, OCEAN SHORES""",2483,"""house""",null,null,771.4,"""M""","""2024-02-06""","""2024-02-09""","""R2""","""R""","""RESIDENCE""","""337/238455"""


In [7]:
geolocator = Nominatim(user_agent="code_for_sydney")

In [24]:
def geocode_address(address):
    try:
        location = geolocator.geocode(address + ", Australia")
        if location:
            return pl.Series([location.latitude, location.longitude])
        return pl.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return pl.Series([None, None])
    finally:
        # Be nice to the API and avoid rate limiting
        time.sleep(0.1)

In [25]:
geocode_address("162 Clarence street Sydney")

""
f64
-33.798564
151.136847


In [20]:
ryde_only = df.filter(pl.col('council_name') == 'RYDE')
ryde_only_2024 = ryde_only.filter(pl.col("settlement_date").str.contains("2024-"))
ryde_only_2024

property_id,download_date,council_name,purchase_price,address,post_code,property_type,strata_lot_number,property_name,area,area_type,contract_date,settlement_date,zoning,nature_of_property,primary_purpose,legal_description
i64,str,str,i64,str,i64,str,i64,str,f64,str,str,str,str,str,str,str
3391609,"""2024-02-19""","""RYDE""",1375000,"""3/6 KULGOA AVE, RYDE""",2112,"""unit""",3,null,null,null,"""2023-12-20""","""2024-02-14""",null,"""R""","""RESIDENCE""","""3/SP80806"""
4170021,"""2024-02-19""","""RYDE""",905000,"""1509/1 NETWORK PL, NORTH RYDE""",2113,"""unit""",218,null,null,null,"""2024-01-18""","""2024-02-15""",null,"""R""","""RESIDENCE""","""218/SP98059"""
4170021,"""2024-02-19""","""RYDE""",650000,"""1702/1 NETWORK PL, NORTH RYDE""",2113,"""unit""",239,null,null,null,"""2024-01-18""","""2024-02-15""",null,"""R""","""RESIDENCE""","""239/SP98059"""
4165471,"""2024-02-19""","""RYDE""",860000,"""1/260 VICTORIA RD, GLADESVILLE""",2111,"""unit""",27,null,169.0,"""M""","""2023-12-28""","""2024-02-09""",null,"""R""","""RESIDENCE""","""27/SP90479"""
868823,"""2024-02-19""","""RYDE""",2590000,"""27 A DARVALL RD, EASTWOOD""",2122,"""house""",null,null,626.0,"""M""","""2023-12-09""","""2024-02-12""","""R2""","""R""","""RESIDENCE""","""2/543885"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3852882,"""2024-01-15""","""RYDE""",923000,"""703/15 CHATHAM RD, WEST RYDE""",2114,"""unit""",137,null,110.0,"""M""","""2023-10-23""","""2024-01-08""",null,"""R""","""RESIDENCE""","""137/SP90616"""
3956155,"""2024-01-15""","""RYDE""",1875000,"""2/647 BLAXLAND RD, EASTWOOD""",2122,"""unit""",2,null,305.0,"""M""","""2023-09-30""","""2024-01-08""",null,"""R""","""RESIDENCE""","""2/SP93317"""
4293060,"""2024-01-15""","""RYDE""",550000,"""505/39 DEVLIN ST, RYDE""",2112,"""unit""",67,null,75.0,"""M""","""2023-11-21""","""2024-01-11""",null,"""R""","""RESIDENCE""","""67/SP100467"""


In [27]:
# Create new columns for latitude and longitude
lat_values = []
lon_values = []

# Loop through each row and extract geocoding information
total = len(ryde_only_2024)
i = 0
for row in ryde_only_2024.rows(named=True):
    addr = f"{row['address']}, {row['post_code']}, {row['council_name']}"
    lat_lon = geocode_address(addr)
    i += 1
    # Extract values from the Series
    lat_values.append(lat_lon[0])
    lon_values.append(lat_lon[1])
    print(f"{i}/{total}")

# Add the new columns to the DataFrame
ryde_only_2024 = ryde_only_2024.with_columns([
    pl.Series("lat", lat_values),
    pl.Series("lon", lon_values)
])

1/514
2/514
3/514
Error geocoding 1/260 VICTORIA RD, GLADESVILLE, 2111, RYDE: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1%2F260+VICTORIA+RD%2C+GLADESVILLE%2C+2111%2C+RYDE%2C+Australia&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
4/514
5/514
6/514
7/514
8/514
9/514
10/514
11/514
12/514
13/514
14/514
15/514
16/514
17/514
18/514
19/514
20/514
21/514
22/514
23/514
24/514
25/514
26/514
27/514


KeyboardInterrupt: 

In [23]:
ryde_only_2024.write_csv("ryde_transactions_2024.csv")

In [30]:
df['address'].unique()

address
str
"""403/18 LILYDALE ST, MARRICKVIL…"
"""42 ST CLAIR ST, BONNELLS BAY"""
"""4 MINDONA ST, LEUMEAH"""
"""6 VINE ST, CHISHOLM"""
"""66 CATALINA WAY, NORTH RICHMON…"
…
"""44 DILLWYNIA DR, GLENMORE PARK"""
"""1103/26 A BELMONT ST, SUTHERLA…"
"""54 HIGH ST, GRETA"""
